<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical

In [30]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# common variables
epochs = 30
showWorking = True
batchSize = 200
dropoutRate = 0.3

In [32]:
# LSTM model
model_lstm = Sequential([
	LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
	Dropout(dropoutRate),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax'),
])
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
useless, accuracy = model_lstm.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)
print("Accuracy of LSTM model on test: ", accuracy)

Epoch 1/30
396/396 [==============================] - 5s 12ms/step - loss: 1.0896 - acc: 0.3510
Epoch 2/30
396/396 [==============================] - 0s 775us/step - loss: 1.0481 - acc: 0.4268
Epoch 3/30
396/396 [==============================] - 0s 784us/step - loss: 1.0130 - acc: 0.4394
Epoch 4/30
396/396 [==============================] - 0s 801us/step - loss: 0.9548 - acc: 0.5732
Epoch 5/30
396/396 [==============================] - 0s 720us/step - loss: 0.8244 - acc: 0.7071
Epoch 6/30
396/396 [==============================] - 0s 690us/step - loss: 0.6177 - acc: 0.7677
Epoch 7/30
396/396 [==============================] - 0s 701us/step - loss: 0.8618 - acc: 0.5859
Epoch 8/30
396/396 [==============================] - 0s 690us/step - loss: 1.1926 - acc: 0.3131
Epoch 9/30
396/396 [==============================] - 0s 729us/step - loss: 1.0696 - acc: 0.4343
Epoch 10/30
396/396 [==============================] - 0s 789us/step - loss: 0.8696 - acc: 0.5884
Epoch 11/30
396/396 [=========

In [33]:
# CNN-LSTM
trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

conv1 = Conv1D(filters=64, kernel_size=5, activation='relu')
conv2 = Conv1D(filters=64, kernel_size=5, activation='relu')
model_lstm_cnn = Sequential([
	TimeDistributed(conv1, input_shape=(None, 20, x_train.shape[2])),
	TimeDistributed(conv2),
	TimeDistributed(Dropout(dropoutRate)),
	TimeDistributed(MaxPooling1D(pool_size=2)),
	TimeDistributed(Flatten()),
	LSTM(200),
	Dropout(dropoutRate),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax')
])
model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
useless, accuracy = model_lstm_cnn.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)
print("Accuracy on test-set for LSTM-CNN: ", accuracy)

Epoch 1/30
396/396 [==============================] - 5s 13ms/step - loss: 1.0860 - acc: 0.3763
Epoch 2/30
396/396 [==============================] - 0s 120us/step - loss: 1.0502 - acc: 0.4268
Epoch 3/30
396/396 [==============================] - 0s 127us/step - loss: 1.0147 - acc: 0.4293
Epoch 4/30
396/396 [==============================] - 0s 122us/step - loss: 0.9455 - acc: 0.6313
Epoch 5/30
396/396 [==============================] - 0s 124us/step - loss: 0.8439 - acc: 0.7146
Epoch 6/30
396/396 [==============================] - 0s 131us/step - loss: 0.6794 - acc: 0.7551
Epoch 7/30
396/396 [==============================] - 0s 133us/step - loss: 0.4993 - acc: 0.8384
Epoch 8/30
396/396 [==============================] - 0s 147us/step - loss: 0.3599 - acc: 0.8914
Epoch 9/30
396/396 [==============================] - 0s 140us/step - loss: 0.2677 - acc: 0.9242
Epoch 10/30
396/396 [==============================] - 0s 135us/step - loss: 0.3957 - acc: 0.8687
Epoch 11/30
396/396 [=========

In [34]:
# LSTM + conv
trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

model_lstm_conv = Sequential([
	ConvLSTM2D(filters=100, kernel_size=(1,5), input_shape=(3, 1, 20, x_train.shape[2])),
	Activation('relu'),
	Dropout(dropoutRate),
	Flatten(),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax')
])
model_lstm_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_conv.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
useless, accuracy = model_lstm_conv.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)
print("Accuracy on test for LSTM-Conv: ", accuracy)

Epoch 1/30
396/396 [==============================] - 5s 14ms/step - loss: 1.0855 - acc: 0.3687
Epoch 2/30
396/396 [==============================] - 0s 281us/step - loss: 1.0362 - acc: 0.4268
Epoch 3/30
396/396 [==============================] - 0s 278us/step - loss: 0.9805 - acc: 0.4268
Epoch 4/30
396/396 [==============================] - 0s 242us/step - loss: 0.8855 - acc: 0.6919
Epoch 5/30
396/396 [==============================] - 0s 262us/step - loss: 0.7536 - acc: 0.8258
Epoch 6/30
396/396 [==============================] - 0s 244us/step - loss: 0.6101 - acc: 0.8157
Epoch 7/30
396/396 [==============================] - 0s 227us/step - loss: 0.4663 - acc: 0.8939
Epoch 8/30
396/396 [==============================] - 0s 224us/step - loss: 0.3652 - acc: 0.9066
Epoch 9/30
396/396 [==============================] - 0s 210us/step - loss: 0.3103 - acc: 0.9015
Epoch 10/30
396/396 [==============================] - 0s 200us/step - loss: 0.2861 - acc: 0.9192
Epoch 11/30
396/396 [=========